# Imports

In [55]:
import sympy

In [56]:
from sympy import *
from sympy.stats import Exponential, density

# Expressing the liquid and ice PDFs as a function of their expected values

In [57]:
# Define the symbol and parameters
x = symbols('x', nonnegative=True)
lambdliq = symbols('lambda_liq', positive=True)
wliq = symbols('w_liq', positive=True)

In [58]:
# Create the exponential distribution
expliq = lambdliq * exp(-lambdliq*x)
# Create the weighted sum
histliq = wliq*DiracDelta(x) + expliq
# Integrate it to normalize it
int_histliq = integrate(histliq, (x, 0, oo))
# Create normalized pdf
pdfliq = histliq/int_histliq

In [59]:
pdfliq

(lambda_liq*exp(-lambda_liq*x) + w_liq*DiracDelta(x))/(w_liq/2 + 1)

In [60]:
# Now reexpress pdfliq as a function of its mean and lambda parameter

# Calculate mean
mu_liq = simplify(integrate(x*pdfliq, (x, 0, oo)))

# Solve for w_liq as a function of mean
mu_l = symbols('mu_l')
wl = solve(mu_liq-mu_l,wliq)

# Reexpress the pdf accordingly
pdfl = (wl[0]*DiracDelta(x) + expliq)/(1+wl[0]/2)

In [62]:
mu_l

mu_l

In [61]:
wl

[-2 + 2/(lambda_liq*mu_l)]

In [8]:
# Finalizing the distributions for liquid and ice
lambice = symbols('lambda_ice', positive=True, real=True)
muice = symbols('mu_ice', positive=True, real=True)
pdfice = pdfl.subs({lambdliq:lambice,mu_l:muice})

lambliq = symbols('lambda_liq', positive=True, real=True)
muliq = symbols('mu_liq', positive=True, real=True)
pdfliq = pdfice.subs({lambice:lambliq,muice:muliq})

In [65]:
pdfl

lambda_liq*mu_l*(lambda_liq*exp(-lambda_liq*x) + (-2 + 2/(lambda_liq*mu_l))*DiracDelta(x))

In [66]:
print(pdfl)

lambda_liq*mu_l*(lambda_liq*exp(-lambda_liq*x) + (-2 + 2/(lambda_liq*mu_l))*DiracDelta(x))


In [10]:
# Testing integral
simplify(integrate(pdfliq,(x,0,oo)))

1

In [11]:
# Testing mean
integrate(x*pdfliq,(x,0,oo))

mu_liq

# Convolving the PDFs to get the PDF of condensed water

In [12]:
# Calculating the convolution of pdfliq and pdfice to get pdfqt
qt = symbols('q_T', nonnegative=True, real=True)
z = symbols('z', real=True)

In [13]:
lambda_ice = symbols('lambda_ice', positive=True, real=True)
mu_ice = symbols('mu_ice', positive=True, real=True)
lambda_liq = symbols('lambda_liq', positive=True, real=True)
mu_liq = symbols('mu_liq', positive=True, real=True)
q_T = symbols('q_T', nonnegative=True, real=True)

## Term by term

In [14]:
convol = pdfliq.subs({x:z})*pdfice.subs({x:(qt-z)})

In [15]:
convol

lambda_ice*lambda_liq*mu_ice*mu_liq*(lambda_ice*exp(-lambda_ice*(q_T - z)) + (-2 + 2/(lambda_ice*mu_ice))*DiracDelta(q_T - z))*(lambda_liq*exp(-lambda_liq*z) + (-2 + 2/(lambda_liq*mu_liq))*DiracDelta(z))

In [16]:
print(convol)

lambda_ice*lambda_liq*mu_ice*mu_liq*(lambda_ice*exp(-lambda_ice*(q_T - z)) + (-2 + 2/(lambda_ice*mu_ice))*DiracDelta(q_T - z))*(lambda_liq*exp(-lambda_liq*z) + (-2 + 2/(lambda_liq*mu_liq))*DiracDelta(z))


In [17]:
a_1 = symbols('a_1', positive=True, real=True)
a_2 = symbols('a_2', positive=True, real=True)
a_3 = symbols('a_3', positive=True, real=True)
a_4 = symbols('a_4', positive=True, real=True)

In [18]:
to_integrate = a_1*exp(-lambda_ice*(q_T - z))*exp(-lambda_liq*z) + a_2*exp(-lambda_ice*(q_T - z))*DiracDelta(z) + a_3*exp(-lambda_liq*z)*DiracDelta(q_T - z) + a_4*DiracDelta(z)*DiracDelta(q_T - z)

In [19]:
to_integrate

a_1*exp(-lambda_ice*(q_T - z))*exp(-lambda_liq*z) + a_2*exp(-lambda_ice*(q_T - z))*DiracDelta(z) + a_3*exp(-lambda_liq*z)*DiracDelta(q_T - z) + a_4*DiracDelta(z)*DiracDelta(q_T - z)

In [20]:
I = integrate( a_1*exp(-lambda_ice*(q_T - z))*exp(-lambda_liq*z),(z,0,q_T))

In [21]:
I.simplify()

Piecewise((a_1*exp(-lambda_liq*q_T)/(lambda_ice - lambda_liq) - a_1*exp(-lambda_ice*q_T)/(lambda_ice - lambda_liq), Ne(lambda_ice, lambda_liq)), (a_1*q_T*exp(-lambda_liq*q_T), True))

**Missing a delta-term here for qt = 0!**

In [22]:
I = a_1*(exp(-lambda_liq*q_T)-exp(-lambda_ice*q_T))/(lambda_ice - lambda_liq)+\
a_2*exp(-lambda_ice*q_T) + a_3*exp(-lambda_liq*q_T)

In [23]:
a_1 = lambda_ice*lambda_liq*mu_liq*mu_ice*lambda_ice*lambda_liq
a_2 = lambda_ice*lambda_liq*mu_liq*mu_ice*2*lambda_ice*(1/(lambda_liq*mu_liq)-1)
a_3 = lambda_ice*lambda_liq*mu_liq*mu_ice*2*lambda_liq*(1/(lambda_ice*mu_ice)-1)

In [24]:
I

a_1*(exp(-lambda_liq*q_T) - exp(-lambda_ice*q_T))/(lambda_ice - lambda_liq) + a_2*exp(-lambda_ice*q_T) + a_3*exp(-lambda_liq*q_T)

In [25]:
I

a_1*(exp(-lambda_liq*q_T) - exp(-lambda_ice*q_T))/(lambda_ice - lambda_liq) + a_2*exp(-lambda_ice*q_T) + a_3*exp(-lambda_liq*q_T)

# Cloud cover

In [43]:
a = symbols('a')

In [44]:
integrate(I,(q_T,a,oo))

-a_1*(-exp(-a*lambda_liq)/lambda_liq + exp(-a*lambda_ice)/lambda_ice)/(lambda_ice - lambda_liq) + a_2*exp(-a*lambda_ice)/lambda_ice + a_3*exp(-a*lambda_liq)/lambda_liq

In [50]:
integrate(I,(q_T,a,oo)).subs({a:-1e6})

-a_1*(-exp(1000000.0*lambda_liq)/lambda_liq + exp(1000000.0*lambda_ice)/lambda_ice)/(lambda_ice - lambda_liq) + a_2*exp(1000000.0*lambda_ice)/lambda_ice + a_3*exp(1000000.0*lambda_liq)/lambda_liq

In [51]:
integrate(I,(q_T,a,oo)).subs({a:1e-6}).simplify()

a_1*(exp(-1.0e-6*lambda_liq)/lambda_liq - exp(-1.0e-6*lambda_ice)/lambda_ice)/(lambda_ice - lambda_liq) + a_2*exp(-1.0e-6*lambda_ice)/lambda_ice + a_3*exp(-1.0e-6*lambda_liq)/lambda_liq

In [52]:
A = symbols('A',real=True)
A = integrate(a_1*(exp(-lambda_liq*q_T)-exp(-lambda_ice*q_T))/(lambda_ice - lambda_liq)+\
a_2*exp(-lambda_ice*q_T) + a_3*exp(-lambda_liq*q_T),(q_T,a,oo)).subs({a:0}).simplify()

In [68]:
print(A)

-3*lambda_ice*lambda_liq*mu_ice*mu_liq + 2*lambda_ice*mu_ice + 2*lambda_liq*mu_liq


In [54]:
a_1 = lambda_ice*lambda_liq*mu_liq*mu_ice*lambda_ice*lambda_liq
a_2 = lambda_ice*lambda_liq*mu_liq*mu_ice*2*lambda_ice*(1/(lambda_liq*mu_liq)-1)
a_3 = lambda_ice*lambda_liq*mu_liq*mu_ice*2*lambda_liq*(1/(lambda_ice*mu_ice)-1)

In [37]:
A

-3*lambda_ice*lambda_liq*mu_ice*mu_liq + 2*lambda_ice*mu_ice + 2*lambda_liq*mu_liq

## Showing that integrals of DIRAC delta are not trustworthy below

In [366]:
I = integrate( a_1*exp(-lambda_ice*(q_T - z))*exp(-lambda_liq*z),(z,0,q_T))

In [368]:
I.simplify()

Piecewise((a_1*exp(-lambda_liq*q_T)/(lambda_ice - lambda_liq) - a_1*exp(-lambda_ice*q_T)/(lambda_ice - lambda_liq), Ne(lambda_ice, lambda_liq)), (a_1*q_T*exp(-lambda_liq*q_T), True))

In [364]:
I = integrate(a_3*exp(-lambda_liq*z)*DiracDelta(q_T - z),(z,0,q_T))

In [365]:
I

-a_3*exp(-lambda_liq*q_T)*Heaviside(-q_T) + a_3*exp(-lambda_liq*q_T)/2

In [295]:
print(to_integrate)

lambda_ice**2*lambda_liq**2*mu_ice*mu_liq*exp(-lambda_ice*q_T)*exp(lambda_ice*z)*exp(-lambda_liq*z) - 2*lambda_ice**2*lambda_liq*mu_ice*mu_liq*exp(-lambda_ice*q_T)*exp(lambda_ice*z)*DiracDelta(z) + 2*lambda_ice**2*mu_ice*exp(-lambda_ice*q_T)*exp(lambda_ice*z)*DiracDelta(z) - 2*lambda_ice*lambda_liq**2*mu_ice*mu_liq*exp(-lambda_liq*z)*DiracDelta(q_T - z) + 4*lambda_ice*lambda_liq*mu_ice*mu_liq*DiracDelta(z)*DiracDelta(q_T - z) - 4*lambda_ice*mu_ice*DiracDelta(z)*DiracDelta(q_T - z) + 2*lambda_liq**2*mu_liq*exp(-lambda_liq*z)*DiracDelta(q_T - z) - 4*lambda_liq*mu_liq*DiracDelta(z)*DiracDelta(q_T - z) + 4*DiracDelta(z)*DiracDelta(q_T - z)


In [309]:
term_1 = lambda_ice**2*lambda_liq**2*mu_ice*mu_liq*exp(-lambda_ice*q_T)*exp(lambda_ice*z)*exp(-lambda_liq*z)
term_1_int = integrate(term_1,(z,0,q_T))

In [310]:
term_1_int

Piecewise((lambda_ice**2*lambda_liq**2*mu_ice*mu_liq/(lambda_ice*exp(lambda_liq*q_T) - lambda_liq*exp(lambda_liq*q_T)) - lambda_ice**2*lambda_liq**2*mu_ice*mu_liq*exp(-lambda_ice*q_T)/(lambda_ice - lambda_liq), Ne(lambda_ice, lambda_liq)), (lambda_ice**2*lambda_liq**2*mu_ice*mu_liq*q_T*exp(-lambda_ice*q_T), True))

In [300]:
print(term_1_int)

Piecewise((-lambda_ice*lambda_liq**2*mu_ice*mu_liq*exp(-lambda_ice*q_T)/(1 - lambda_liq/lambda_ice), (lambda_ice - lambda_liq < 0) & Ne(lambda_liq/lambda_ice, 1)), (Integral(lambda_ice**2*lambda_liq**2*mu_ice*mu_liq*exp(-lambda_ice*q_T)*exp(lambda_ice*z)*exp(-lambda_liq*z), (z, 0, oo)), True))


In [303]:
term1I = (-lambda_ice*lambda_liq**2*mu_ice*mu_liq*exp(-lambda_ice*q_T)/(1 - lambda_liq/lambda_ice)).simplify()

In [304]:
term1I

-lambda_ice**2*lambda_liq**2*mu_ice*mu_liq*exp(-lambda_ice*q_T)/(lambda_ice - lambda_liq)

## All at once

In [339]:
pdfqt = integrate(pdfliq.subs({x:z})*pdfice.subs({x:(qt-z)}),(z,0,qt))
pdfqt = pdfqt.expand().simplify()

In [340]:
pdfqt

Piecewise((lambda_liq**2*mu_liq*(-lambda_ice**2*mu_ice*(1 - exp(q_T*(lambda_ice - lambda_liq)))*exp(lambda_liq*q_T) + (lambda_ice - lambda_liq)*(2*lambda_ice*mu_ice*Heaviside(-q_T) - lambda_ice*mu_ice - 2*Heaviside(-q_T) + 1)*exp(lambda_ice*q_T))*exp(-q_T*(lambda_ice + lambda_liq))/(lambda_ice - lambda_liq), Ne(lambda_ice, lambda_liq)), (lambda_liq**2*mu_liq*(lambda_ice**2*mu_ice*q_T*exp(lambda_liq*q_T) + (2*lambda_ice*mu_ice*Heaviside(-q_T) - lambda_ice*mu_ice - 2*Heaviside(-q_T) + 1)*exp(lambda_ice*q_T))*exp(-q_T*(lambda_ice + lambda_liq)), True))

In [322]:
print(pdfqt)

Piecewise((lambda_liq**2*mu_liq*(-lambda_ice**2*mu_ice*(1 - exp(q_T*(lambda_ice - lambda_liq)))*exp(lambda_liq*q_T) + (lambda_ice - lambda_liq)*(2*lambda_ice*mu_ice*Heaviside(-q_T) - lambda_ice*mu_ice - 2*Heaviside(-q_T) + 1)*exp(lambda_ice*q_T))*exp(-q_T*(lambda_ice + lambda_liq))/(lambda_ice - lambda_liq), Ne(lambda_ice, lambda_liq)), (lambda_liq**2*mu_liq*(lambda_ice**2*mu_ice*q_T*exp(lambda_liq*q_T) + (2*lambda_ice*mu_ice*Heaviside(-q_T) - lambda_ice*mu_ice - 2*Heaviside(-q_T) + 1)*exp(lambda_ice*q_T))*exp(-q_T*(lambda_ice + lambda_liq)), True))


In [326]:
pdfqt_tmp = (lambda_liq**2*mu_liq*(-lambda_ice**2*mu_ice*(1 - exp(q_T*(lambda_ice - lambda_liq)))*exp(lambda_liq*q_T) + (lambda_ice - lambda_liq)*(2*lambda_ice*mu_ice*Heaviside(-q_T)*0 - lambda_ice*mu_ice - 2*Heaviside(-q_T)*0 + 1)*exp(lambda_ice*q_T))*exp(-q_T*(lambda_ice + lambda_liq))/(lambda_ice - lambda_liq))

In [331]:
pdfqt_tmp

lambda_liq**2*mu_liq*(-lambda_ice**2*mu_ice*(1 - exp(q_T*(lambda_ice - lambda_liq)))*exp(lambda_liq*q_T) + (lambda_ice - lambda_liq)*(-lambda_ice*mu_ice + 1)*exp(lambda_ice*q_T))*exp(-q_T*(lambda_ice + lambda_liq))/(lambda_ice - lambda_liq)

In [345]:
pdfqt_tmp.factor().expand().simplify().collect(q_T)

lambda_liq**2*mu_liq*(-lambda_ice**2*mu_ice*exp(lambda_liq*q_T) + lambda_ice*lambda_liq*mu_ice*exp(lambda_ice*q_T) + lambda_ice*exp(lambda_ice*q_T) - lambda_liq*exp(lambda_ice*q_T))*exp(-q_T*(lambda_ice + lambda_liq))/(lambda_ice - lambda_liq)

In [346]:
a = symbols('a',positive=True)
CC = integrate(pdfqt_tmp,(q_T,a,oo))

In [350]:
CC.collect(mu_ice)

-lambda_liq**2*mu_liq*(-lambda_ice*exp(-a*lambda_liq)/lambda_liq + mu_ice*(-lambda_ice*exp(-a*lambda_liq) + lambda_ice*exp(-a*lambda_ice)) + exp(-a*lambda_liq))/(lambda_ice - lambda_liq)

In [353]:
CC.subs({a:0}).simplify()

lambda_liq*mu_liq